In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/data_YesBank_StockPrices.csv")
data

,Date,Open,High,Low,Close
0,Jul-05,13.00,14.00,11.25,12.46
1,Aug-05,12.58,14.88,12.55,13.42
2,Sep-05,13.48,14.87,12.27,13.30
3,Oct-05,13.20,14.47,12.40,12.99
4,Nov-05,13.35,13.88,12.88,13.41
...,...,...,...,...,...
180,Jul-20,25.60,28.30,11.10,11.95
181,Aug-20,12.00,17.16,11.85,14.37
182,Sep-20,14.30,15.34,12.75,13.15
183,Oct-20,13.30,14.01,12.11,12.42


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[['Open', 'High', 'Low', 'Close']])

# Create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i+seq_length]
        label = data[i+seq_length-1]
        sequences.append((seq, label))
    return sequences

seq_length = 10
sequences = create_sequences(data_scaled, seq_length)

# Build the LSTM model
model = keras.Sequential([
    layers.LSTM(50, activation='relu', input_shape=(seq_length, data_scaled.shape[1])),
    layers.Dense(data_scaled.shape[1])
])
model.compile(optimizer='adam', loss='mse')

# Convert sequences to arrays
X = np.array([seq for seq, label in sequences])
y = np.array([label for seq, label in sequences])

# Train the model without printing progress
model.fit(X, y, epochs=50, batch_size=64, verbose=0)

# Make predictions
predictions = model.predict(X)

# Inverse transform to get actual values
predictions_actual = scaler.inverse_transform(predictions)
y_actual = scaler.inverse_transform(y)

# Create a DataFrame to hold results
results = pd.DataFrame({
    'Date': data['Date'].values[seq_length - 1:],
    'Actual_Close': y_actual[:, 3],
    'Predicted_Close': predictions_actual[:, 3]
})

# Print the results
print(results)

6/6 [==============================] - 1s 4ms/step
       Date  Actual_Close  Predicted_Close
0    Apr-06         19.49        21.551077
1    May-06         18.03        22.199060
2    Jun-06         15.62        21.980539
3    Jul-06         16.16        21.600349
4    Aug-06         17.84        21.565084
..      ...           ...              ...
171  Jul-20         11.95        27.863132
172  Aug-20         14.37        23.846621
173  Sep-20         13.15        21.349699
174  Oct-20         12.42        19.677326
175  Nov-20         14.67        18.725208

[176 rows x 3 columns]
